In [24]:
from common.alpha import Alpha
from common.td import TD

import datetime
import pandas as pd
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

pd.set_option("display.max_columns", 100)
init_notebook_mode(connected=True)

In [2]:
alpha = Alpha()

td_client = TD()

In [3]:
# Goal: Pull minute data for the last 5 years for specific ETF stocks
data = td_client.get_price_history(symbol='DGAZ', period_type='year', period=5, frequency_type='daily', frequency=1, needExtendedHoursData=False)

In [4]:
# Converting dict to pandas df
df = pd.DataFrame.from_dict(data)
df.head()

,candles,symbol,empty
0,"{'open': 343.0, 'high': 361.01, 'low': 342.0, ...",DGAZ,False
1,"{'open': 361.0, 'high': 380.0, 'low': 352.1, '...",DGAZ,False
2,"{'open': 361.0, 'high': 372.0, 'low': 355.0, '...",DGAZ,False
3,"{'open': 370.0, 'high': 385.0, 'low': 364.0, '...",DGAZ,False
4,"{'open': 384.0, 'high': 387.0, 'low': 375.0, '...",DGAZ,False


In [5]:
# Expanding 'candles' col to multiple columns
df = pd.concat([df.drop(['candles'], axis=1), df['candles'].apply(pd.Series)], axis=1)
df.head()

,symbol,empty,open,high,low,close,volume,datetime
0,DGAZ,False,343.0,361.01,342.0,352.0,189042.0,1.404709e+12
1,DGAZ,False,361.0,380.00,352.1,356.0,164559.0,1.404796e+12
2,DGAZ,False,361.0,372.00,355.0,371.0,42768.0,1.404882e+12
3,DGAZ,False,370.0,385.00,364.0,384.0,104711.0,1.404968e+12
4,DGAZ,False,384.0,387.00,375.0,377.0,34522.0,1.405055e+12


In [7]:
# Create 'date' col from 'datetime'. Converting from epoch time from milliseconds to seconds, then to date
df['date'] = df['datetime'].apply(lambda x: datetime.datetime.fromtimestamp(x/1000.0).strftime('%Y-%m-%d'))
df.head()

,symbol,empty,open,high,low,close,volume,datetime,date
0,DGAZ,False,343.0,361.01,342.0,352.0,189042.0,1.404709e+12,2014-07-07
1,DGAZ,False,361.0,380.00,352.1,356.0,164559.0,1.404796e+12,2014-07-08
2,DGAZ,False,361.0,372.00,355.0,371.0,42768.0,1.404882e+12,2014-07-09
3,DGAZ,False,370.0,385.00,364.0,384.0,104711.0,1.404968e+12,2014-07-10
4,DGAZ,False,384.0,387.00,375.0,377.0,34522.0,1.405055e+12,2014-07-11


In [23]:
trace = go.Candlestick(x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])
dgaz_plot = [trace]
iplot(dgaz_plot, filename='charts/dgaz_test')

In [48]:
INCREASING_COLOR = '#17BECF'
DECREASING_COLOR = '#7F7F7F'

In [49]:
data = [ dict(
    type = 'candlestick',
    open = df['open'],
    high = df['high'],
    low = df['low'],
    close = df['close'],
    x = df['date'],
    yaxis = 'y2',
    name = 'Historic Data',
    increasing = dict( line = dict( color = INCREASING_COLOR ) ),
    decreasing = dict( line = dict( color = DECREASING_COLOR ) ),
) ]

layout=dict()

fig = dict( data=data, layout=layout )

In [50]:
fig['layout'] = dict()
fig['layout']['plot_bgcolor'] = 'rgb(250, 250, 250)'
fig['layout']['xaxis'] = dict( rangeselector = dict( visible = True ) )
fig['layout']['yaxis'] = dict( domain = [0, 0.2], showticklabels = False )
fig['layout']['yaxis2'] = dict( domain = [0.2, 0.8] )
fig['layout']['legend'] = dict( orientation = 'h', y=0.9, x=0.3, yanchor='bottom' )
fig['layout']['margin'] = dict( t=40, b=40, r=40, l=40 )

In [51]:
rangeselector=dict(
    visibe = True,
    x = 0, y = 0.9,
    bgcolor = 'rgba(150, 200, 250, 0.4)',
    font = dict( size = 13 ),
    buttons=list([
        dict(count=1,
             label='reset',
             step='all'),
        dict(count=1,
             label='1 yr',
             step='year',
             stepmode='backward'),
        dict(count=3,
            label='1 month',
            step='month',
            stepmode='backward'),
        dict(count=1,
            label='1 day',
            step='day',
            stepmode='backward'),
        dict(step='all')
    ]))
    
fig['layout']['xaxis']['rangeselector'] = rangeselector

In [52]:
# Add EMA line


mv_y = movingaverage(df['close'])
mv_x = list(df['date'])

fig['data'].append( dict( x=mv_x, y=mv_y, type='scatter', mode='lines', 
                         line = dict( width = 1 ),
                         marker = dict( color = '#E377C2' ),
                         yaxis = 'y2', name='Moving Average' ) )

In [53]:
colors = []

for i in range(len(df['close'])):
    if i != 0:
        if df['close'].iloc[i] > df['close'].iloc[i-1]:
            colors.append(INCREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    else:
        colors.append(DECREASING_COLOR)

In [54]:
fig['data'].append( dict( x=df['date'], y=df['volume'],                         
                         marker=dict( color=colors ),
                         type='bar', yaxis='y', name='Volume' ) )

In [55]:
iplot(fig, filename='charts/dgaz_test2', validate=False)